# O Autocarro Heurístico

<img src="Figuras\LondonBus.gif" alt="Drawing" style="width: 250px;"/>

## Introdução
Considere o problema do autocarro da avaliação contínua 1 em que um condutor de autocarro com capacidade limitada, sai de uma estação e vai apanhar e largar passageiros de acordo com as reservas, regressando à estação. Ele conhece o mapa da região que corresponde a uma grelha quadrada, sabendo quais as ruas pedonais, quais as de sentido único e o tipo de tráfego em cada rua. Está a par também do tempo médio que leva a atravessar cada rua, que pode ser de tráfego ligeiro, médio ou intenso e tem uma estimativa do tempo que leva cada passageiro a entrar e a sair nas paragens. O seu objectivo é planear o trajecto que leva menos tempo.

## Objetivos
Terão de implementar uma função heurística consistente, que iremos descrever e ilustrar já a seguir, e que garante a solução óptima se usarmos o algoritmo A* na versão em grafo.

## Função Heurística

### Descrição Geral da Função Heurística 
Uma função heurística é sempre uma estimativa do custo mínimo de cada estado ao objectivo. No problema do autocarro, o custo é o tempo estimado que demora a fazer um trajecto que parte da estação, apanhando e largando clientes, regressando à estação, numa grelha com possibilidades de restrições nos sentidos da circulação das ruas. Como sabem, as ruas podem ser de 3 tipos em termos de intensidade de tráfego, sendo o tempo médio de circulação numa rua dependente da respectiva intensidade e tendo em consideração o tempo médio de paragem para carregar e descarregar pessoas.

Para que esta heurística seja admissível, é preciso subestimar sempre o custo mínimo real. Para começar vamos ignorar as ruas pedonais e de sentido único e considerar que todas as ruas são bidireccionais. Notem que os trajectos ortogonais entre 2 pontos serão mínimos se todas as ruas forem bidireccionais. Para não sobrestimarmos o tempo real de um trajecto, assumimos que todas as ruas demoram o tempo da rua mais rápida. Assim, devido à existência de ruas pedonais e de sentido único e de ruas mais demoradas do que a mais rápida, o trajecto real óptimo pode ser ainda mais demorado.

Vamos considerar que a heurística corresponde ao tempo de largar todos os passageiros no autocarro e apanhar e largar todos os que estejam fora do autocarro mais o tempo do trajecto maior possível por cliente. Se pensarmos que o autocarro tem um só cliente, calculamos o melhor trajecto para ele. O trajecto mais demorado entre todos os trajectos dos clientes será uma estimativa por baixo do tempo do trajecto real. Temos sempre que satisfazer o cliente mais demorado e fazer o trajecto óptimo para o levar ao destino e regressar à estação ou para o apanhar, levar ao destino e regressar à estação.

Pensamos que esta heurística para além de admissível é também consistente. Para isso é necessário, considerando $X,Y$ estados do espaço do problema ligados por um arco ou acção: $\forall _{X \rightarrow Y} h(Y)-h(X) \leq custo(X,Y)$ e todos os estados finais terão de ter um valor heurístico 0, i.e., $\forall _{G} goal(G) \Rightarrow h(G)=0$,  

A seguir, vamos fazer uma descrição mais em pormenor da função heurística pedida que ilustraremos com *bons* exemplos.

### Descrição Detalhada da Função Heurística 
Vamos considerar duas situações para a definição da função heurística:

* O autocarro está vazio e basta regressar à estação. Nesse caso, a função devolve o tempo mínimo entre a estação e a posição do autocarro.
* Ainda há pessoas para largar ou pessoas para recolher e largar. Nesse caso vamos relaxar o problema e considerar apenas o mais demorado dos trajectos mínimos de cada cliente. Chamamos de mínimo porque assumimos que todas as ruas são bidireccionais e com a duração mais rápida. Ao tempo mínimo maior somamos o tempo gasto a carregar e descarregar todas pessoas fora do autocarro e a descarregar todas as pessoas dentro do autocarro.

O tempo de cada pessoa que faz parte do estado é determinado da seguinte maneira: 
- se estiver no autocarro é o tempo mínimo que leva até ao seu destino + o tempo mínimo que leva do seu destino até à estação;
- se for uma pessoa fora do autocarro é o tempo mínimo que leva do autocarro até a sua localização + o tempo mínimo que leva até o largar + o tempo mínimo para regressar à estação.

Vamos chamar de tempo mínimo de uma posição X a uma posição Y, a distância de Manhatan de X a Y multiplicada pelo tempo associado à travessia da rua mais rápida.

A distância de Manhatan entre dois pontos dá-nos o trajecto ortogonal mínimo entre dois pontos, em termos do número de ruas. Notem que a distância de manhatan entre dois pontos é a soma da diferença entre os valores absolutos das coordenadas respectivas: $manhatan((x1,y1),(x2,y2))=|x1-x2|+|y1-y2|$

### Exemplo 1
Vamos imaginar que o mundo é $5 \times 5$, que não há ruas de tráfego ligeiro, que a estação está em $(0,0)$ e que temos 3 clientes, Dubenka, Lou e Gagarin, estão todos em $(1,2)$, em que Lou quer ir para $(3,3)$, e Gagarin e Dubenka vão ambos para $(2,3)$. O tempo de paragem por passageiro é de $10s$ e as ruas de tráfego médio demoram $300s$ a serem percorridas.

Nesse caso, o valor da função heurística seria

para o caso de Lou: $manhatan((0,0),(1,2))+manhatan((1,2),(3,3))+manhatan((3,3),(0,0))=3+3+6=12$

para o caso de Gagarin:$manhatan((0,0),(1,2))+manhatan((1,2),(2,3))+manhatan((2,3),(0,0))=3+2+5=10$

para o caso de Dubenka, o mesmo que Gagarin, pois a localização e destino são os mesmos: $10$

O tempo de paragem será de $2 \times 2 \times 10s=40s$

A função heurística devolverá $max(12,10) \times 300s + 40s=3660s$


### Exemplo 2
Vamos imaginar uma grelha $25 \times 25$, o autocarro na posição $(3,3)$, a estação em $(1,2)$, uma rua com intensidade ligeira é atravessada em média em $50s$ e o tempo de paragem por passageiro é estimado em $10s$, havendo ruas de tráfego ligeiro.

e que temos dentro do autocarro:
* Valdemar que quer sair na posição $(20,20)$
* Zulu que quer sair na posição $(2,2)$
    
e fora do autocarro:
* Francis que está em $(10,10)$ e quer sair em $(4,4)$
* Berta que está em $(0,0)$ e quer sair em $(7,0)$
    
Neste caso as distâncias dos trajectos mínimos de cada um dos passageiros seriam:

* Distância do trajecto de Valdemar$=34+37=71$
(34 de distância do autocarro para o destino + 37 do destino para a estação)
* Distância do trajecto de Zulu$=2+1=3$
* Distância do trajecto de Francis=$14+12+5=31$
(14 a distância do autocarro a Francis, depois 12 até ao seu destino e 5 de regresso à estação) 
* Distância do trajecto de Berta$=6+7+8=21$

Os trajectos seriam multiplicados pelo tempo para atravessar uma rua ligeira que é de $50s$, obtendo os tempos mínimos de cada um dos passageiros.

Bom, o tempo mínimo maior: $71\times50s=3550s$

O tempo para carregar Francis e Berta e descarregar as 4 pessoas: $(2+2\times2) \times 10=60s$

O valor da função heurística seria $3550s+60s=3610s$

### Exemplo 3
Para a mesma grelha $25 \times 25$, a estação é em $(1,2)$, i.e., em tudo semelhante ao mundo anterior, mas em que o autocarro está na estação com o Francis e a Berta e ninguém lá fora à espera
```python
capacidade=5
estacao=(1,2)
pedonais={((4,3),(4,4))}
sentidoUnico={((3,3),(3,2)),((1,2),(2,2))}
intenso={((3,2),(3,3)),(3,1),(3,2),((2,0),(2,1))}
ligeiro={((2,1),(2,2))}
tempos={'ligeiro': 50, 'médio': 300, 'intenso': 1000}
tempoPassageiro=10
clientes={'Jim':((1,1),(3,3)),'Valdemar':((3,2),(20,20)),'Francis':((10,10),(4,4)),'Berta':((0,0),(7,0)),'Zulu':((1,3),(2,2))}
```

Para esse caso, a função heurística devolve $1740s$.

### Exemplo 4
```python
dim=6
capacidade=5
estacao=(1,2)
pedonais={((4,3),(4,4)),((0,0),(0,1)),((4,4),(4,5))}
sentidoUnico={((3,3),(3,2)),((1,2),(2,2))}
intenso={((3,2),(3,3)),(3,1),(3,2),((2,0),(2,1))}
ligeiro={((2,1),(2,2)),((2,2),(2,3)),((4,0),(4,1))}
tempos={'ligeiro': 60, 'médio': 200, 'intenso': 900}
tempoPassageiro=10
clientes={'Jim':((1,1),(3,3)),'Valdemar':((3,2),(0,3)),'Francis':((3,4),(5,0)),'Berta':((0,0),(3,3)),'Zulu':((1,3),(4,2))}

mundoX=MundoBus(dim,estacao,capacidade,pedonais,sentidoUnico,intenso,ligeiro,tempos,tempoPassageiro,clientes)

Para este problema o A* devolve a seguinte solução e custo

['s', ([], ['Jim']), 's', 'o', ([], ['Berta']), 'e', 'n', 'n', 'n', ([], ['Zulu']), 'e', 'e', (['Berta', 'Jim'], []), 'n', ([], ['Francis']), 's', 'e', 's', (['Zulu'], []), 's', 's', 'e', (['Francis'], []), 'o', 'n', 'o', 'n', ([], ['Valdemar']), 'o', 'n', 'o', 'o', (['Valdemar'], []), 's', 'e']
4880
```

## A classe OBus
Fornecemos uma implementação da formulação do problema do autocarro, para ser igual para todos, a qual devem adicionar o código da função heurística, a que chamámos de `h1`. Existe um método chamado de prettyAction que pega numa acção e devolve-a de um modo mais adequado para ser avaliada automaticamente. Não modifica as *strings* em {"n","s","e","0"}, mas as acções de apanhar/largar são tuplos $(Larga,Agarra)$ mas em que Larga e Agarra em vez de serem conjuntos, são listas ordenadas pelos identificadores dos respectivos clientes. Temos de fazer a correcção automática e como não se garante que os conjuntos saiam sempre iguais quando se faz `print`, teremos de usar listas ordenadas. Existe também um método prettyPlan que recebe uma lista de acções e devolve-a numa forma que seja avaliada automaticamente.

In [5]:
import itertools
from copy import deepcopy
from elEstadoMundo import *
from searchPlus import *

# Mundo por defeito
estacao=(0,0)
pedonais={((4,3),(4,4))}
sentidoUnico={((3,3),(3,2)),((1,2),(2,2))}
intenso={((3,2),(3,3))}
ligeiro={}
tempos={'ligeiro': 100, 'médio': 300, 'intenso': 1000}
tempoPassageiro=10
clientes={'Lou':((1,2),(3,3)),'Gagarin':((1,2),(2,3)),'Dubenka':((1,2),(2,3))}

mundoDef=MundoBus(5,estacao,2,pedonais,sentidoUnico,intenso,ligeiro,tempos,tempoPassageiro,clientes)

class OBus(Problem):

    
    tabMovs={'e':(1,0),'n':(0,1),'o':(-1,0),'s':(0,-1)}
    
    def __init__(self, initial=None,goal=None, mundo=mundoDef):
        """O objectivo é o tuplo ordenado por ordem crescente e o estado inicial é uma permutação de range(1, n+1).
        """
        self.mundo=mundo
        self.initial = EstadoBus(mundo.estacao,set(mundo.clientes.keys()),set())
        self.goal=EstadoBus(mundo.estacao,set(),set())

    def move(pos,accao):
        x,y=pos
        incX,incY=OBus.tabMovs[accao]
        return ((x+incX) ,(y+incY))
    
    def actions(self, state):
        """ As acções aplicáveis
        """
        out=[]
        for a in ["e","n","o","s"]:
            x,y=state.pos
            incX,incY=OBus.tabMovs[a]
            new=((x+incX),(y+incY))
            if 0<=(x+incX)< self.mundo.dim and 0<=(y+incY)< self.mundo.dim and \
               (state.pos,new) not in self.mundo.pedonais and \
               (new,state.pos) not in self.mundo.pedonais and \
               (new,state.pos) not in self.mundo.sentidoUnico:
                out.append(a)
        ## all that should drop will drop
        droppingNum=0
        drops=set()
        for d in state.destinos:
            if self.mundo.clientes[d][1]==state.pos:
                drops.add(d)
                droppingNum+=1   

        ## the most that can be picked up will be picked up
        ## that depends on the bus capacity
        allPossiblePickups=[p for p in state.pickup if self.mundo.clientes[p][0]==state.pos]
        allPossiblePickups.sort()  # ordena

        freePlacesNum=self.mundo.capacidade-(len(state.destinos)-droppingNum)

        pickupNum=min(freePlacesNum,len(allPossiblePickups))
        if pickupNum!=0:
            acts=[(drops,set(x)) for x in (itertools.combinations(allPossiblePickups, pickupNum))]
        elif drops!=set():
            acts=[(drops,set())]
        else:
            acts = []
        return acts + out

    def result(self, estado, accao):
        """Acções de movimento e de pickup and drop. 
            "n", "s", "l", "o" e (pickupSet,DestinosSet)
        """

        if type(accao)==str:
            newPos=OBus.move(estado.pos,accao)
            return EstadoBus(newPos,estado.pickup,estado.destinos)
        else:
            drop,pickingUp=accao
            
            # dropping all people that go out of the bus here and 
            # the pickupers will be destinies
            moreDestinos=pickingUp.copy()
            newDestinos=(estado.destinos.copy() - drop).union(moreDestinos)
            
            # update pickup
            newPickup=deepcopy(estado.pickup)-pickingUp
            newPos=estado.pos

        return EstadoBus(newPos,newPickup,newDestinos)
    
    
    def path_cost(self, c, state1,action,next_state):
        """ As acções de movimento têm custos associados ao tempo para a travessia que depende do tipo de rua
            As acções de apanhar/largar pessoas têm um custo que corresponde ao número de pessoas que saiem e entram
            a multiplicar pelo tempo médio que demora cada pessoa a entrar ou sair.
        """
        if type(action)==str:
            if (state1.pos,next_state.pos) in self.mundo.ligeiro or (next_state.pos,state1.pos) in self.mundo.ligeiro:
                custo=self.mundo.tempos['ligeiro']
            elif (state1.pos,next_state.pos) in self.mundo.intenso or (next_state.pos,state1.pos) in self.mundo.intenso:
                custo=self.mundo.tempos['intenso']
            else:
                custo=self.mundo.tempos['médio']
            return c+custo
        return c+(len(action[0])+len(action[1]))*self.mundo.tempoPassageiro
    
    def prettyAction(self,a):
        a1,a2=a
        a1=list(a1)
        a1.sort()
        a2=list(a2)
        a2.sort()
        return ((a1,a2))

    def prettyPlan(self,plan):
        prettyP=[]
        for a in plan:
            if type(a)==str:
                prettyP.append(a)
            else:
                prettyP.append(self.prettyAction(a))
        return prettyP

    def dist(x1,y1,x2,y2):
        return abs(x1 - x2) + abs(y1 - y2)
    
    def h1(self,node):
        if len(node.state.pickup) == 0 and len(node.state.destinos) == 0: # se nao houver mais passageiros
            return OBus.dist(self.mundo.estacao[0], node.state.pos[0], self.mundo.estacao[1], node.state.pos[1]) # retorna distancia a estacao

        else:
            tempos = list()
            if len(node.state.pickup) != 0:
                for p in node.state.pickup:
                    c = self.mundo.clientes.get(p)
                    # distancia do autocarro ao cliente + distancia do cliente ao destino + distancia do destino a estacao
                    tempos.append(OBus.dist(node.state.pos[0], c[0][0], node.state.pos[1], c[0][1]) + \
                                  OBus.dist(c[0][0], c[1][0], c[0][1], c[1][1]) + \
                                  OBus.dist(self.mundo.estacao[0], c[1][0], self.mundo.estacao[1], c[1][1]))
            if len(node.state.destinos) != 0:
                for d in node.state.destinos:
                    c = self.mundo.clientes.get(d)
                    # distancia do autocarro ao destino + distancia do destino a estacao
                    tempos.append(OBus.dist(node.state.pos[0], c[1][0], node.state.pos[1], c[1][1]) + \
                                  OBus.dist(self.mundo.estacao[0], c[1][0], self.mundo.estacao[1], c[1][1]))
                                  
            # calcular o tempo minimo maior
            m = max(tempos)
            # calcular o tempo do tipo de estrada
            if len(self.mundo.ligeiro) != 0:
                t = self.mundo.tempos.get("ligeiro")
            else: #instenso??
                t = self.mundo.tempos.get("médio")

            # calcular tempo de movimento
            t *= m

            # calcular tempo de entradas e saidas
            e = (len(node.state.pickup) + len(node.state.destinos)) * self.mundo.tempoPassageiro

            return e + t
capacidade=5
estacao=(1,2)
pedonais={((4,3),(4,4))}
sentidoUnico={((3,3),(3,2)),((1,2),(2,2))}
intenso={((3,2),(3,3)),(3,1),(3,2),((2,0),(2,1))}
ligeiro={((2,1),(2,2))}
tempos={'ligeiro': 50, 'médio': 300, 'intenso': 1000}
tempoPassageiro=10
clientes={'Jim':((1,1),(3,3)),'Valdemar':((3,2),(20,20)),'Francis':((10,10),(4,4)),'Berta':((0,0),(7,0)),'Zulu':((1,3),(2,2))}
p=OBus()
print(p.h1(Node(p.initial)))
    
   


1230


## Nota
* A função heurística valerá $0.625$

## Ficheiros anexos e imports
O ficheiro `elEstadoMundo.py` continua a ter que ser importado, juntamente com `utils.py` e `searchPlus.py`, os quais não devem ser alterados. Podem importar os módulos que desejarem.

## Deadline
Esta avaliação contínua é **individual** e terá **correcção automática** sendo submetida através de um quiz na página da disciplina. Para essa correcção automática iremos disponibilizar alguns testes que serão visíveis e outros testes invisíveis. Podem fazer *check* e *submit* as vezes que quiserem, sem quaisquer penalizações, e a nota final será a da tentativa mais elevada sendo a última tentativa submetida automaticamente no fecho do Quiz.

O prazo é Terça-feira, $1 \space de \space Novembro,\space às\space 23:59$.

<img src="Figuras\old-ukrainian-bus-2773390.JPG" alt="Drawing" style="width: 350px;"/>